# 0. Import needed Libraries

In [ ]:
# the 3 main libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from eofs.xarray import Eof as eof
# librairies to do nicer plots...
from matplotlib.patches import Rectangle    # only to draw a rectangle

# 1. SST:

In [ ]:
data = xr.open_dataset('sst.mnmean.nc')
data_mask = xr.open_dataset('lsmask.nc')
mask = data_mask.mask                          # get the DataArray from the Dataset
mask = mask.squeeze()                          # get rid of the time dimension in the variable mask

In [ ]:
weights = np.cos( np.deg2rad(mask.lat) )
weights = mask*weights

### Detrend SST

In [ ]:
linfit = data.sst.polyfit('time', 1)
trend = xr.polyval(coord=data.time, coeffs=linfit.polyfit_coefficients)   # SST trend
sst_detrend = data.sst - trend.values + data.sst.mean(dim='time')         # detrended SST

### Detrended interannual anomaly

In [ ]:
sstbymth = sst_detrend.groupby("time.month")
mthclim = sstbymth.mean("time")       # detrended climatological months (seasonal cycle)
sstanom = sstbymth - mthclim          # detrended interannual annomaly
sstanom = sstanom.rename('sstanom')   # change variable name

### Compute nino3.4 (5S-5N and 170-120W) index

In [ ]:
weights_nino34 = weights.sel(lon=slice(360-169.5,360-120.5), lat=slice(4.5,-4.5))
nino34_index = sstanom.weighted(weights_nino34).mean(dim=('lon','lat'))
nino34_index = nino34_index.rename('nino34')   # change variable name
nino34_index.plot()

### Create an [Eof](https://ajdawson.github.io/eofs/latest/api/eofs.xarray.html#eofs.xarray.Eof) object

In [ ]:
solver = eof(sstanom, weights=weights)

### Explained variance by the different EOFs/PCs couple

In [ ]:
solver.varianceFraction(neigs=10).plot.step(where='mid')

###  Plot the first 2 EOFs, PCs and Variance Fraction:  

In [ ]:
# eof = linear_regression_coef(PC,sstanom) = Cov(PC,sstanom)/Var(PC)
# pcscaling=1 (default) --> Var(PC) = 1 --> eof = Cov(PC,sstanom)
eofs = solver.eofsAsCovariance(neofs=2, pcscaling=1)      # beware of syntaxe: A C in eofsAsCovariance
pcs  = solver.pcs(npcs=2, pcscaling=1)
varfrac = solver.varianceFraction(neigs=2)                # beware of syntaxe: F in varianceFraction

# create a 4 pannels figure
fig, axes = plt.subplots(2,2,figsize=(10, 5),constrained_layout=True)
# plot EOF1 and PC1
eofs.sel(mode=0).plot(ax=axes[0,0], cbar_kwargs={'label': '°C for 1 std of PC'})
axes[0,0].set_title('EOF 1: '+str(int(varfrac.values[0]*100))+'%') 
pcs.sel(mode=0).plot(ax=axes[1,0])
axes[1,0].set_title('PC 1')
# plot EOF2 and PC2
eofs.sel(mode=1).plot(ax=axes[0,1], cbar_kwargs={'label': '°C for 1 std of PC'})
axes[0,1].set_title('EOF 2: '+str(int(varfrac.values[1]*100))+'%')
pcs.sel(mode=1).plot(ax=axes[1,1])
axes[1,1].set_title('PC 2')

### correlation of the first 2 PC with Nino3.4

In [ ]:
pcs  = solver.pcs(npcs=2, pcscaling=1)
xr.corr(pcs, nino34_index, dim='time').data

# 2. MSL:

In [ ]:
data_msl = xr.open_dataset('msl_era5.nc')
data_msl

In [ ]:
dtarr_msl = data_msl.msl

In [ ]:
dtarr_msl.mean(dim='time').plot()

In [ ]:
mask_msl = dtarr_msl.isel(time=0)                              # trick to quickly create a xarray 
mask_msl.data = np.full(np.shape(dtarr_msl.isel(time=0)), 1)   # fill the data with only 1
mask_msl = mask_msl.squeeze()                                  # remove dimension equal to 1

In [ ]:
weights_msl = np.cos(np.deg2rad(dtarr_msl.lat))
weights_msl = mask_msl*weights_msl * np.full(np.shape(dtarr_msl.isel(time=0)), 1)
weights_msl.plot()

### Detrend MSL

In [ ]:
linfit = dtarr_msl.polyfit('time', 1)
trend = xr.polyval(coord=dtarr_msl.time, coeffs=linfit.polyfit_coefficients)   # MSL trend
msl_detrend = dtarr_msl - trend.values + dtarr_msl.mean(dim='time')            # detrended MSL

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(6,3), constrained_layout=True)
a = dtarr_msl.groupby('time.year').mean(dim='time')      # yearly mean
a.weighted(weights_msl).mean(dim=('lon','lat')).plot(ax=axes[0], label="org")
a = trend.groupby('time.year').mean(dim='time')         # yearly mean
a.weighted(weights_msl).mean(dim=('lon','lat')).plot(ax=axes[0], label="trend")
axes[0].legend()
axes[0].set_title('With trend')
#
a = msl_detrend.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights_msl).mean(dim=('lon','lat')).plot(ax=axes[1], label="detrend")
axes[1].legend()
axes[1].set_title('Without trend')

### Detrended interannual anomaly

In [ ]:
mslbymth = msl_detrend.groupby("time.month")
mthclim = mslbymth.mean("time")       # detrended climatological months (seasonal cycle)
mslanom = mslbymth - mthclim          # detrended interannual annomaly
mslanom = mslanom.rename('mslanom')   # change variable name

In [ ]:
a = dtarr_msl.weighted(weights_msl).mean(dim=('lon','lat'))
clim = a.groupby("time.month").mean("time")   # climatological months (seasonal cycle)
anom = a.groupby("time.month") - clim         # interannual annomaly
anom.plot(label="org")
#
mslanom.weighted(weights_msl).mean(dim=('lon','lat')).plot(label="detrended")
plt.legend()

### Interannual standard deviation

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5), constrained_layout=True)
msl_detrend.std(dim='time').where(mask_msl == 1.).plot(ax=axes[0],vmin=0,vmax=1000,cmap='YlGnBu')
axes[0].set_title('MSL detrend STD')
mslanom.std(dim='time').where(mask_msl == 1.).plot(ax=axes[1],vmin=0,vmax=1000,cmap='YlGnBu')
axes[1].set_title('MSL detrend + anom STD')
# overplot nino3.4 rectangle (170W-120W, 5S-5N)
ax = plt.gca()                                                              # Get the current reference
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='r',fill=False)   # Create a Rectangle patch
ax.add_patch(rect)                                                          # Add the patch to the Axes

### MSL anomalies regressed onto Nino3.4 SST

In [ ]:
dmin = np.max([mslanom.time.min().data, nino34_index.time.min().data])
dmax = np.min([mslanom.time.max().data, nino34_index.time.max().data])
mslanom2 = mslanom.sel(time=slice(dmin,dmax))
nino34_2 = nino34_index.sel(time=slice(dmin,dmax))
timesave = mslanom2.time
mslanom2.coords["time"] = (("time"), nino34_2.data)  # redefine 'time' coordinates with nino34 time series
linfit = mslanom2.polyfit('time', 1)                     # compute the regression with this new "time" 
mslanom2.coords["time"] = (("time"), timesave.data)     # put back original time

mslreg = linfit.polyfit_coefficients.isel(degree=0)
mslcor = xr.corr(mslanom2, nino34_2, dim='time')                      

#  create a 2 pannels figure
fig, axes = plt.subplots(1,2,figsize=(14, 4))
mslreg.where(mask_msl == 1.).plot(ax=axes[0])                                # plot the regression coefficient
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[0].add_patch(rect)                                                     # overlay 
axes[0].set_title('Regression coefficient')
mslcor.where(mask_msl == 1.).plot(ax=axes[1])                                # plot the correlation
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[1].add_patch(rect)                                                     # overlay 
axes[1].set_title('Correlation')
axes[1].set_xlabel('lon')
axes[1].set_ylabel('lat')

### Create an [Eof](https://ajdawson.github.io/eofs/latest/api/eofs.xarray.html#eofs.xarray.Eof) object

In [ ]:
#solver = eof(mslanom, weights=weights_msl)
solver = eof(mslanom.sel(lat=slice(-20., 20.)), weights=weights_msl.sel(lat=slice(-20., 20.)))

### Explained variance by the different EOFs/PCs couple

In [ ]:
solver.varianceFraction(neigs=10).plot.step(where='mid')

###  Plot the first 2 EOFs, PCs and Variance Fraction:  

In [ ]:
# eof = linear_regression_coef(PC,sstanom) = Cov(PC,sstanom)/Var(PC)
# pcscaling=1 (default) --> Var(PC) = 1 --> eof = Cov(PC,sstanom)
eofs = solver.eofsAsCovariance(neofs=2, pcscaling=1)      # beware of syntaxe: A C in eofsAsCovariance
pcs  = solver.pcs(npcs=2, pcscaling=1)
varfrac = solver.varianceFraction(neigs=2)                # beware of syntaxe: F in varianceFraction

# create a 4 pannels figure
fig, axes = plt.subplots(2,2,figsize=(10, 5),constrained_layout=True)
# plot EOF1 and PC1
eofs.sel(mode=0).plot(ax=axes[0,0], cbar_kwargs={'label': '°C for 1 std of PC'})
axes[0,0].set_title('EOF 1: '+str(int(varfrac.values[0]*100))+'%') 
pcs.sel(mode=0).plot(ax=axes[1,0])
axes[1,0].set_title('PC 1')
# plot EOF2 and PC2
eofs.sel(mode=1).plot(ax=axes[0,1], cbar_kwargs={'label': '°C for 1 std of PC'})
axes[0,1].set_title('EOF 2: '+str(int(varfrac.values[1]*100))+'%')
pcs.sel(mode=1).plot(ax=axes[1,1])
axes[1,1].set_title('PC 2')

### correlation of the first 2 PC with Nino3.4

In [ ]:
pcs  = solver.pcs(npcs=2, pcscaling=1)
xr.corr(pcs, nino34_index, dim='time').data